In [1]:
import numpy as np
import sys
sys.executable
from os.path import join
from smpl_webuser.serialization import load_model
from fitting.util import write_simple_obj, safe_mkdir
import trimesh
import sklearn
import pyrender
from smpl_torch import SMPLModel
from sklearn.metrics import mean_squared_error
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

learning_rate = 0.0002
gamma         = 0.98

In [2]:
def reward(model, mesh):
    return - sum(sum((mesh.vertices - model)**2))# minimize distance between meshes

In [3]:
#load the pytorch model
pose_size = 15
beta_size = 400

pose = torch.from_numpy((np.zeros(pose_size))).type(torch.double)
betas = torch.from_numpy((np.zeros(beta_size))).type(torch.float64)
trans = torch.from_numpy(np.zeros(3)).type(torch.float64)
model = SMPLModel()
result = model(betas, pose, trans)

In [4]:
#RL code
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.rewards = []
        self.states = []
        
        self.fc1 = nn.Linear(15, 128)
        self.fc2 = nn.Linear(128, 15)
       # self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
    
    def put_data(self, item, states):
        self.rewards.append(item)
        self.states.append(states)

In [5]:
def main():
    pose_size = 15
    beta_size = 400
    
    coeffs = torch.tensor(np.zeros((1, 15)).astype('float64'), requires_grad = True)
    pose = torch.from_numpy((np.zeros(pose_size))).type(torch.float64)
    betas = torch.from_numpy((np.zeros(beta_size))).type(torch.float64)
    trans = torch.from_numpy(np.zeros(3)).type(torch.float64)
    model = SMPLModel()
    result = model(betas, pose, trans)
    print(pose.size)
    print(coeffs)
    pi = Policy()
    score = 0 
    optimizer = optim.Adam(pi.parameters(), lr=learning_rate)
    for n_epi in range(10000):
        #complete episode
        R = 0
        for name in sorted(glob.glob('/home/michalislazarou/PhD/VOCA/voca/animation_output/*.obj')):
            print(name)
            for i in range (1000):
                obj_load = trimesh.load_mesh(name)
                coeffs = pi(coeffs.float())
                model = SMPLModel()
                result = model(betas, coeffs.double(), trans)
                
                #r = reward(result, obj_load)
                #pi.put_data(r, coeffs)
                #coeffs = pose
                # training my net
                #R = r + gamma * R
                
                error = (result - torch.from_numpy(obj_load.vertices))
                loss = torch.sum(torch.abs(result - torch.from_numpy(obj_load.vertices)))
               # print(loss.shape)
                optimizer.zero_grad()
                loss.backward(retain_graph=True) 
                optimizer.step()
                if i%100 == 0:
                    print(coeffs)
                    mesh = trimesh.Trimesh(result.detach(), model.faces)
                    mesh.show(viewer = 'gl')
                    obj_load.show(viewer = 'gl')
                #score += r

#pi.train_net()
#mesh = trimesh.Trimesh(result, model.faces)
#mesh.show(viewer = 'gl')

In [ ]:
main()

<built-in method size of Tensor object at 0x7f458412d460>
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64, requires_grad=True)
/home/michalislazarou/PhD/VOCA/voca/animation_output/00000.obj
tensor([[0.0000, 0.0000, 0.0054, 0.0000, 0.0550, 0.0000, 0.0160, 0.0000, 0.0688,
         0.0000, 0.0729, 0.0843, 0.0483, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5245,
         0.0000, 0.0000, 0.2712, 0.3392, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
